In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-27T10:44:08.270147+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [3]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')

In [4]:
# Get the token from Openai/Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['GOOGLE_CSE_ID'] = GOOGLE_CSE_ID

In [5]:
import langchain
import faiss #pip install faiss-cpu
%watermark --iversions 

langchain: 0.0.336
faiss    : 1.7.4



## What is AutoGPT

Unlike traditional language models that generate text based solely on the input they receive, AutoGPT combines a set of tools and a memory system to interact with its environment, retrieve past interactions, and generate more informed responses.

AutoGPT distinguishes itself with three critical features. First, its internet connectivity provides real-time access to information, bolstering its research abilities. Second, its planning capability enables it to generate and pursue sub-tasks that contribute to the completion of a main task. Lastly, it has the power to execute tasks, even to the extent of initiating other AI agents. While these features offer huge potential, they also present challenges, such as inadvertent task looping or prematurely assuming task completion. 

Through internet browser plugins and other forms of access, AutoGPT gathers the information needed to execute tasks. 

An external memory serves as its context-aware module, enabling it to understand its current situation, generate new tasks, and even self-correct if necessary. As a result, AutoGPT operates dynamically, continually re-prioritizing tasks based on the context and situation.

### Tools Setup

We initialize different tools that the AI agent can use to complete tasks. In our case, the tools are Search, WriteFileTool, and ReadFileTool. The Search tool utilizes a GoogleSearchAPIWrapper to fetch real-time information from the internet, which can be employed for questions about current events or queries that need up-to-date information. The WriteFileTool and ReadFileTool manage file-related tasks. These tools are collected into a list that will be later passed to the agent. 

In [5]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

#Set up the tools
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="Useful for when you need to answer questions about current events. You should ask targeted questions",
        return_direct=True
    ),
    WriteFileTool(),
    ReadFileTool(),
]

### Agent Memory Setup

For the memory, we create the FAISS vector DB (but any other vector DB would work similarly), an efficient similarity search, and clustering of dense vectors. This is paired with an InMemoryDocstore instance for storing documents in memory and an OpenAIEmbeddings model for creating embeddings of the queries. These tools are crucial for the agent's remembering and retrieving past interactions.

AutoGPT has been designed to operate over longer periods. AutoGPT has incorporated a retrieval-based memory system that functions over intermediate agent steps to do that. 

This memory performs a semantic search across embeddings using the vector DB. While such retrieval-based memory is a part of LangChain, it was traditionally used for user and agent interactions, not agent and tools. AutoGPT's new adaptation represents a significant shift in how this memory system is applied.

In [12]:
# Set up the memory
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
#from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")
#model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {'device': 'cpu'}
#embeddings_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
embedding_size = 1536

import faiss
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


### Setting up the Model and AutoGPT

Here we initialize the AutoGPT agent, giving it a name ("Jim") and a role ("Assistant"). We also supplied it with the tools and memory systems that were established in the previous steps. The language model being used here is ChatOpenAI, which is set to have a temperature of 0 (indicating deterministic responses).

In [13]:
# Set up the model and AutoGPT
from langchain_experimental.autonomous_agents import AutoGPT
from langchain.chat_models import ChatOpenAI
#from gpt4allj.langchain import GPT4AllJ

agent = AutoGPT.from_llm_and_tools(
    ai_name="Jim",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    #llm=GPT4AllJ(model=workingFolder+'models\\ggml-gpt4all-j-v1.3-groovy.bin',temperature=0),
    memory=vectorstore.as_retriever()
)

# Set verbose to be true
agent.chain.verbose = True

### Running an Example

Finally, we provided an example task for the AutoGPT agent. This task ("Provide an analysis of the major historical events that led to the French Revolution") is complex and requires the agent to utilize its tools and memory system effectively to generate a response.

The agent takes some minutes to generate the final answer, but we get a peek into all the intermediate computations thanks to having set the verbose variable to True.

There are a lot of intermediate computations and the output is very long.

In [14]:
task = "Provide an analysis of the major historical events that led to the French Revolution"

agent.run([task])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Jim, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Provide an analysis of the major historical events that led to the French Revolution


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: Useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. wr


> Finished chain.
{
  "thoughts": {
    "text": "I have gathered some information about the major historical events that led to the French Revolution. Now, I should save this information to a file for future reference.",
    "reasoning": "By saving the information to a file, I can easily access it later and refer to it when analyzing and providing a comprehensive analysis of the events.",
    "plan": "- Use the 'write_file' command to save the gathered information to a file.\n- Choose a descriptive file name that reflects the content.\n- Ensure that the file is saved in a location where it can be easily accessed and organized.",
    "criticism": "I need to make sure that I save the information accurately and in a well-organized manner. It's important to use clear and concise language when writing the file.",
    "speak": "I will now save the gathered information about the major historical events that led to the French Revolution to a file for future reference."
  },
  "command": {
   


> Finished chain.
{
  "thoughts": {
    "text": "Now that I have saved the information about the major historical events that led to the French Revolution, I should read the file to ensure that the information was saved accurately.",
    "reasoning": "By reading the file, I can verify that the information was saved correctly and ensure that I have access to the accurate details of the events.",
    "plan": "- Use the 'read_file' command to read the file containing the information about the major historical events.\n- Verify that the content of the file matches the information I gathered.\n- Take note of any discrepancies or errors that need to be addressed.",
    "criticism": "I need to be thorough in checking the file to ensure that the information is accurate. It's important to pay attention to details and cross-reference the information with reliable sources if necessary.",
    "speak": "I will now read the file containing the information about the major historical events that led 


> Finished chain.
{
  "thoughts": {
    "text": "Now that I have read the file and verified the accuracy of the information, I can proceed with analyzing the major historical events that led to the French Revolution.",
    "reasoning": "By analyzing the events, I can provide a comprehensive analysis of the causes and factors that contributed to the French Revolution.",
    "plan": "- Review the information from the file and identify the major events that played a significant role in the French Revolution.\n- Take notes on each event and its significance.\n- Analyze the events to identify common themes, causes, and consequences.",
    "criticism": "I need to ensure that my analysis is based on accurate information and supported by historical evidence. It's important to avoid biases and present a balanced perspective.",
    "speak": "I will now analyze the major historical events that led to the French Revolution to provide a comprehensive analysis."
  },
  "command": {
    "name": "wri


> Finished chain.
{
  "thoughts": {
    "text": "Now that I have analyzed the major historical events that led to the French Revolution and written the analysis to a file, I can move on to the next step: providing a summary of the analysis.",
    "reasoning": "By summarizing the analysis, I can provide a concise overview of the key events and their significance in the French Revolution.",
    "plan": "- Review the analysis and identify the main points and key takeaways.\n- Condense the information into a clear and concise summary.\n- Ensure that the summary captures the essence of the analysis and highlights the most important events and their impact.",
    "criticism": "I need to be careful not to oversimplify the analysis in the summary. It's important to strike a balance between brevity and providing enough information to convey the significance of the events.",
    "speak": "I will now provide a summary of the major historical events that led to the French Revolution."
  },
  "com


> Finished chain.
{
  "thoughts": {
    "text": "Now that I have provided a summary of the major historical events that led to the French Revolution, I can proceed with the final step: reflecting on the significance and impact of the revolution.",
    "reasoning": "By reflecting on the revolution, I can analyze its long-term effects, its influence on subsequent events, and its historical significance.",
    "plan": "- Review the analysis and summary of the major events.\n- Consider the political, social, and economic changes brought about by the revolution.\n- Reflect on the impact of the revolution on France and the world, including its influence on subsequent revolutions and the rise of Napoleon Bonaparte.\n- Identify the key lessons and insights that can be drawn from the French Revolution.",
    "criticism": "I need to ensure that my reflection is well-supported by historical evidence and avoids generalizations or oversimplifications. It's important to provide a balanced perspecti


> Finished chain.
{
  "thoughts": {
    "text": "Now that I have reflected on the significance and impact of the French Revolution and written the reflection to a file, I have completed all the objectives. It's time to finish.",
    "reasoning": "By completing all the objectives, I have fulfilled the task of providing an analysis of the major historical events that led to the French Revolution.",
    "plan": "- Review the files containing the analysis, summary, and reflection to ensure they are accurate and comprehensive.\n- Consider any additional steps or tasks that may be necessary to further enhance the analysis.\n- Prepare a final response to indicate the completion of the objectives.",
    "criticism": "I need to ensure that all the files are well-organized and contain the necessary information. It's important to double-check for any errors or omissions.",
    "speak": "I have completed all the objectives and provided an analysis of the major historical events that led to the Fr

'I have completed all the objectives and provided an analysis of the major historical events that led to the French Revolution. Thank you for your assistance!'

### To make it understand better...

The first part of the chain result is

This is the prompt that AutoGPT sends to the LLM for a text continuation. From it, we see:

    That role-prompting is used at the beginning, having an autonomous assistant called Jim.
    The goal that the assistant should pursue, i.e., “Provide an analysis of the major historical events that led to the French Revolution.”
    A set of constraints explicitly explain to the LLM that it has limited memory and that memories are saved into txt files, from which they can also be retrieved.
    A set of commands that the assistant can issue, i.e. (1) “search” to look for external knowledge using a search engine, (2) “write_file” to write content into a file (for storing memories), (3) “read_file” to read content from a file (for retrieving memories) and (4) “Finish” to return the final result and stop the computations.
    The assistant can use a set of high-level resources, like Internet access and an LLM agent, to perform single tasks.
    A set of instructions about continuously refining the assistant plan.
    A response format that the assistant should conform to when answering. Notice that the response format “forces” the LLM into explicitly writing its thinking, reasoning, and a devised plan (i.e., a bullet point list of steps to reach the aforementioned goal). Then, the agent criticizes the plan (i.e., explains what it needs to be careful of) and writes in the “speak” field a natural language explanation of the next action it’s going to do from its plan. All these fields explicitly lead the LLM to think about the next step to do and eventually output a command. 
    Last, the prompt also contains the current time and date and a list of similar past events (which is now empty but won’t be empty in the successive interactions with the assistant).

Let’s see how the agent’s output is to that prompt. Here the output continues:

Here, the agent wrote the output in the expected JSON format. Reading through the “text” and “reasoning,” we see what the agent has thought about before devising the “plan.” Then, the plan is criticized in the “criticism” field, and a natural language explanation is written in the “speak” field. Last, the agent selected as command the “search” command, with the "Key events that led to the French Revolution" value for the “tool_input” parameter.

Once the “search” tool returns its answer, we see the next prompt created by AutoGPT. We’ll show mainly the parts of the prompt which are different from the previous prompt.

In the line starting with “System: This reminds you of these events from your past:,” this time we see a non-empty list containing the LLM output from the previous interaction with the assistant, whose input and output are then added after the line “Human: Determine which next command to use and respond using the format specified above:” in a format similar to a few-shot example.

The complete AutoGPT execution with this goal lasted for several minutes, and we won’t show all its intermediate steps here. The last prompt inputted to the assistant contains the following:

***AI: {***

***"thoughts": {***

***"text": "Now that I have summarized the major historical events that led to the French Revolution, I should analyze their significance and draw some conclusions about the revolution's impact on world history. I should start by organizing my thoughts and outlining my analysis.",***

***"reasoning": "Organizing my thoughts and outlining my analysis will help me to stay focused and ensure that I cover all the key points in my analysis. It will also help me to identify any gaps in my knowledge or areas where I need to do further research.",***

***"plan": "- Organize my thoughts and outline my analysis of the significance of the French Revolution\n- Identify any gaps in my knowledge or areas where I need to do further research",***

***"criticism": "I need to make sure that my analysis is well-supported by evidence and that I am accurately representing the historical events that led to the French Revolution. I should also be careful not to oversimplify the complex factors that contributed to the outbreak of the revolution.",***

***"speak": "I will organize my thoughts and outline my analysis of the significance of the French Revolution, and identify any gaps in my knowledge or areas where I need to do further research."***

***},***

***"command": {***

***"name": "write_file",***

***"args": {***

***"file_path": "french_revolution_analysis.txt",***

***"text": "Analysis of the Significance of the French Revolution:\\\\n\\\\nThe French Revolution was a watershed moment in world history that had a profound impact on the political, social, and economic landscape of Europe and beyond. Some of the key factors that contributed to the significance of the French Revolution include:\\\\n\\\\n- The rise of nationalism: The French Revolution helped to popularize the idea of nationalism, which would go on to play a major role in the political and social movements of the 19th and 20th centuries.\\\\n- The spread of liberal democracy: The French Revolution helped to spread the idea of liberal democracy, which would become the dominant political system in the Western world.\\\\n- The end of feudalism: The French Revolution marked the end of the feudal system in France and paved the way for the rise of capitalism and industrialization.\\\\n- The impact on international relations: The French Revolution had a major impact on international relations, leading to a series of wars and conflicts that would shape the course of European history for decades to come.\\\\ n\\\\n In conclusion, the French Revolution was a pivotal moment in world history that had a profound impact on the political, social, and economic landscape of Europe and beyond. By analyzing the significance of the French Revolution, we can gain a better understanding of the forces that have shaped the modern world.",***

***"append": false***

***}***

 ***}***

 ***}***

***System: Command write_file returned: File written successfully to french_revolution_analysis.txt. Human: Determine which next command to use, and respond using the format specified above:***

This prompt shows that the last executed command wrote the final overview into the “french_revolution_analysis.txt” file. Then, the assistant produced its final output.

As we can see, based on the output Jim, the AI assistant, has performed well, managing to work on three distinct and detailed files. 

Jim’s performance highlights several key capabilities:

    Research and Analysis: The assistant has demonstrated good research and analysis skills. It has managed to delve into the historical context, key events, and the long-term impact of the French Revolution, presenting them in an organized and comprehensible manner.
    Writing and Summarization: The agent displayed good writing skills. It has effectively synthesized complex historical concepts into clear and concise summaries, enabling better understanding for any reader, irrespective of their background knowledge.
    Planning and Workflow Management: It has shown a structured approach in managing the tasks. It completed its research, wrote summaries, and planned for review and presentation, all while maintaining a smooth workflow and ensuring the proper organization and storage of information.
    Autonomy: We are able to see that Jim operated independently, without any user assistance, demonstrating their ability to handle tasks from beginning to end.

The results showcase its capabilities in historical research, analysis, summarization, planning, and information management.